Install requirements for saving model as HDF5

In [1]:
!pip install -q pyyaml h5py

Code for preparing Kaggle and downloading dataset

In [ ]:
!mkdir .kaggle
!touch .kaggle/kaggle.json

api_token = {"username":"{YOUR_USERNAME}","key":"{YOUR_KEY}"}

import json
import zipfile
import os
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 /content/.kaggle/kaggle.json
!mv .kaggle /root/
!kaggle datasets download -d sohaibalam67/corn-disease

 89% 89.0M/100M [00:02<00:00, 44.9MB/s]
100% 100M/100M [00:02<00:00, 44.3MB/s] 


Extract dataset and removing unnessecary files

In [ ]:
!unzip corn-disease.zip
!rm -rf corn
!rm -rf sample_data/

Additional : use this code to mount your google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Model main code

In [ ]:
import tensorflow as tf
import os
import PIL
import matplotlib.pyplot as plt
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow.keras.layers as layers
NUM_OF_CLASSES = 4
NUM_OF_PICS = 3852
BATCH_SIZE = 32
IW = 256
IH = 256
dataset_dir = "/content/Corn/"
gray_leaf_dataset = "/content/Corn/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/"
rust_dataset = "/content/Corn/Corn_(maize)___Common_rust_/"
northern_leaf_dataset="/content/Corn/Corn_(maize)___Common_rust_/"
healthy_dataset = "/content/Corn/Corn_(maize)___healthy/"
PIL.Image.open("/content/Corn/Corn_(maize)___healthy/07317e94-df27-4c29-bd69-6c5e54a0457d___R.S_HL 7998 copy.jpg")
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir,validation_split=0.25,subset="training",seed=8569,image_size=(IH,IW),batch_size=BATCH_SIZE)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir,validation_split=0.25,subset="validation",seed=8569,image_size=(IH,IW),batch_size=BATCH_SIZE)


checkpoint_filepath = 'checkpoints/mdl.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)



model = tf.keras.Sequential([
                             tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
                             tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                             tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(IH, IW, 3)),
                             #tf.keras.layers.Conv2D(16,3,padding="same",activation="relu"),
                             #tf.keras.layers.MaxPool2D(),
                             tf.keras.layers.Conv2D(32,3,padding="same",activation="relu"),
                             tf.keras.layers.MaxPool2D(),
                             tf.keras.layers.Conv2D(64,3,padding="same",activation="relu"),
                             tf.keras.layers.MaxPool2D(),
                             tf.keras.layers.Conv2D(128,3,padding="same",activation="relu"),
                             tf.keras.layers.MaxPool2D(),
                             tf.keras.layers.Conv2D(256,3,padding="same",activation="relu"),
                             tf.keras.layers.MaxPool2D(),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(512,activation="relu"),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(512,activation="relu"),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(4)
                             

])

model.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=["accuracy"])
EPOCHS = 8
model.fit(train_dataset,validation_data=val_dataset,epochs=EPOCHS)
model.save("mdl.h5")

Found 3852 files belonging to 4 classes.
Using 2889 files for training.
Found 3852 files belonging to 4 classes.
Using 963 files for validation.
Epoch 1/8
91/91 [==============================] - 13s 140ms/step - loss: 0.9210 - accuracy: 0.5701 - val_loss: 0.5059 - val_accuracy: 0.8536
Epoch 2/8
91/91 [==============================] - 13s 140ms/step - loss: 0.4077 - accuracy: 0.8183 - val_loss: 0.2276 - val_accuracy: 0.8972
Epoch 3/8
91/91 [==============================] - 13s 139ms/step - loss: 0.3020 - accuracy: 0.8712 - val_loss: 0.2019 - val_accuracy: 0.9117
Epoch 4/8
91/91 [==============================] - 12s 137ms/step - loss: 0.2557 - accuracy: 0.8941 - val_loss: 0.3180 - val_accuracy: 0.8930
Epoch 5/8
91/91 [==============================] - 12s 136ms/step - loss: 0.3111 - accuracy: 0.8643 - val_loss: 0.2077 - val_accuracy: 0.9294
Epoch 6/8
91/91 [==============================] - 12s 136ms/step - loss: 0.2428 - accuracy: 0.9003 - val_loss: 0.1870 - val_accuracy: 0.9263
Epo

Summary of model

In [ ]:
mymodel = tf.keras.models.load_model("mdl.h5")
mymodel.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_24 (Rescaling)     (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_118 (MaxPoolin (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_119 (MaxPoolin (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_120 (MaxPoolin (None, 32, 32, 64)      

Additional: Code for converting model to Lite for using on smartphones, embedded systems, etc

In [ ]:
model = tf.keras.models.load_model('/path/to/')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("lite_mdl.tflite", "wb").write(tflite_model)